# 0) Import
Cambia file di input.

In [ ]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
today = date.today()

uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

input = pd.read_excel('Input/Raw/Ricerca F24.xlsx')
input = clean(input)

# 1) Città presenti in uffici ma non in uffici_ut

In [ ]:
mapping = {'albenga':'savona',
 'chiavenna':'sondrio',
 'cossato':'biella',
 'macomer':'nuoro',
 'isili':'nuoro',
 'chiaromonte': 'lagonegro',
 'orzinuovi' : 'brescia',
 'ovada': 'acquiterme',
 'pavullo nel frignano':'modena',
 'ponte san pietro':'bergamo',
 'romano di lombardia':'bergamo',
 'san benedetto del tronto':'ascoli piceno',
 'teano':'caserta',
 'venezia 2 - mestre' : 'venezia',
 'venosa':'potenza',
 'vimercate':'monza',
 'acerra' : 'casoria'
 }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )

def find_code_for_foro(df1, df2):
    results = []
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        matching_rows = df1[df1['office'].str.contains(foro)]
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()


input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

Se quello qua sotto ha lunghezza 0, non modificare nulla sennò controllare le città in sezioni territoriali:

In [ ]:
input[input['code_office'].isna()]

# 2) Prendiamo gli enti corrispondenti all'ufficio_ut

In [ ]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}

def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

input['code_ente'] = input.apply(get_codes, axis=1)
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))

df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=['nr.decreto', 'year.decreto', 'nr.rg', 'year.rg','nr.repertorio','year.repertorio','nr.crono','year.crono','nr.rgeppt','year.rgeppt','nr.repppt','year.repppt'], 
                  var_name='type', 
                  value_name='value')

df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

input.columns.name = None
input = input.reset_index(drop=True)
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [ ]:
input.to_excel('Input/Cleaned/input_v1.xlsx')